In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.5 pyspark-shell'

In [2]:
import pyspark
import os
import re

In [3]:
DATA_PATH = '../../madrid_sim/'

In [4]:
files = [f for f in os.listdir(DATA_PATH) if (os.path.isfile(os.path.join(DATA_PATH, f)) and re.match('\d_MNO_MNO1.csv', f[-14:]))]

In [5]:
sc = pyspark.SparkContext()

In [6]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [7]:
sqlContext = pyspark.SQLContext(sc)

In [8]:
from pyspark.sql.types import *

schema = StructType([StructField("t", IntegerType(), True), 
                     StructField("AntennaId", IntegerType(), True), 
                     StructField("EventCode", IntegerType(), True),
                     StructField("PhoneId", IntegerType(), True), 
                     StructField("x", FloatType(), True),
                     StructField("y", FloatType(), True),
                     StructField("TileId", IntegerType(), True)
                   ])

In [9]:
main_df = sqlContext.read.csv(pyspark.SparkFiles.get(files[0]), header=True, schema=schema)
for i in range (1, len(files)):
    sc.addFile(os.path.join(DATA_PATH, files[i]))
    main_df = main_df.union(sqlContext.read.csv(DATA_PATH + files[i], header=True ,inferSchema=True,schema=schema))

In [10]:
main_df.show()

+---+---------+---------+-------+---------+---------+------+
|  t|AntennaId|EventCode|PhoneId|        x|        y|TileId|
+---+---------+---------+-------+---------+---------+------+
|  0|      101|        0|    273|600216.44|649458.44|   248|
|  1|      101|        1|    273|599767.75| 650111.9|   432|
|  3|      101|        0|    299| 600098.7| 649273.8|   193|
|  4|      101|        1|    299| 599999.6| 649391.2|   219|
|  3|      103|        0|    273| 600292.8| 650705.7|   600|
|  4|      103|        1|    273|600195.75| 649919.0|   383|
| 38|      103|        0|    253|600225.94| 650540.0|   545|
| 39|      103|        2|    253|600225.94| 650540.0|   545|
| 40|      103|        2|    253|600225.94| 650540.0|   545|
| 41|      103|        2|    253|600225.94| 650540.0|   545|
| 42|      103|        2|    253|600225.94| 650540.0|   545|
| 43|      103|        2|    253|600225.94| 650540.0|   545|
| 44|      103|        2|    253|600225.94| 650540.0|   545|
| 45|      103|        2

In [11]:
main_df = main_df.select([c for c in main_df.columns if c in ['t','PhoneId','x','y']])

In [12]:
main_df = main_df.orderBy('t', ascending=True)

In [13]:
print("Number of partitions: {}".format(main_df.rdd.getNumPartitions()))

Number of partitions: 51


In [19]:
from pyspark.sql.functions import *
windowedt = main_df.groupBy('t').count()

In [22]:
windowedt.orderBy('t').show()

+---+-----+
|  t|count|
+---+-----+
|  0|   22|
|  1|   34|
|  2|   35|
|  3|   32|
|  4|   36|
|  5|   31|
|  6|   31|
|  7|   29|
|  8|   31|
|  9|   26|
| 10|   29|
| 11|   30|
| 12|   27|
| 13|   30|
| 14|   29|
| 15|   28|
| 16|   31|
| 17|   30|
| 18|   32|
| 19|   30|
+---+-----+
only showing top 20 rows



In [41]:
main_df.toJSON().map(lambda j: json.loads(j)).collect() #NON faux 

[{'t': 0, 'PhoneId': 273, 'x': 600216.44, 'y': 649458.44},
 {'t': 0, 'PhoneId': 253, 'x': 600714.94, 'y': 649097.25},
 {'t': 0, 'PhoneId': 234, 'x': 601514.94, 'y': 650068.4},
 {'t': 0, 'PhoneId': 241, 'x': 601462.7, 'y': 648964.1},
 {'t': 0, 'PhoneId': 242, 'x': 601462.7, 'y': 648964.1},
 {'t': 0, 'PhoneId': 268, 'x': 600564.44, 'y': 652729.3},
 {'t': 0, 'PhoneId': 283, 'x': 602103.56, 'y': 649742.2},
 {'t': 0, 'PhoneId': 284, 'x': 602103.56, 'y': 649742.2},
 {'t': 0, 'PhoneId': 230, 'x': 601129.06, 'y': 653193.94},
 {'t': 0, 'PhoneId': 259, 'x': 600203.06, 'y': 651730.06},
 {'t': 0, 'PhoneId': 257, 'x': 600901.56, 'y': 651774.9},
 {'t': 0, 'PhoneId': 289, 'x': 600384.44, 'y': 650135.94},
 {'t': 0, 'PhoneId': 296, 'x': 600995.3, 'y': 652785.56},
 {'t': 0, 'PhoneId': 297, 'x': 600995.3, 'y': 652785.56},
 {'t': 0, 'PhoneId': 245, 'x': 600110.94, 'y': 651829.06},
 {'t': 0, 'PhoneId': 246, 'x': 600110.94, 'y': 651829.06},
 {'t': 0, 'PhoneId': 276, 'x': 601136.56, 'y': 649742.94},
 {'t': 0

In [50]:
main_df

DataFrame[t: int, PhoneId: int, x: float, y: float]

In [54]:
main_df = main_df.withColumn('x', main_df.x/1000).withColumn('y', main_df.y/1000)

+---+-------+-----------+---------+
|  t|PhoneId|          x|        y|
+---+-------+-----------+---------+
|  0|    270|  600.00875|650833.25|
|  0|    246|600.1109375|651829.06|
|  0|    280|599.9173125|648764.94|
|  0|    296|600.9953125|652785.56|
|  0|    245|600.1109375|651829.06|
|  0|    299| 600.431375| 648956.5|
|  0|    241|601.4626875| 648964.1|
|  0|    259|600.2030625|651730.06|
|  0|    289|600.3844375|650135.94|
|  0|    297|600.9953125|652785.56|
|  0|    268|600.5644375| 652729.3|
|  0|    242|601.4626875| 648964.1|
|  0|    276|601.1365625|649742.94|
|  0|    273|600.2164375|649458.44|
|  0|    234|601.5149375| 650068.4|
|  0|    284|602.1035625| 649742.2|
|  0|    230|601.1290625|653193.94|
|  0|    283|602.1035625| 649742.2|
|  0|    257|600.9015625| 651774.9|
|  0|    253|600.7149375|649097.25|
+---+-------+-----------+---------+
only showing top 20 rows



In [ ]:
query = df \
  .selectExpr("CAST(t AS STRING) AS key", "to_json(struct(*)) AS value") \
  .writeStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "host1:port1,host2:port2") \
  .option("topic", "antennas") \
  .start()

In [48]:
import json
main_df_intermediary = main_df.where(main_df.t == 0)
main_df_intermediary.toJSON().map(lambda j: json.loads(j)).collect()

[{'t': 0, 'PhoneId': 273, 'x': 600216.44, 'y': 649458.44},
 {'t': 0, 'PhoneId': 253, 'x': 600714.94, 'y': 649097.25},
 {'t': 0, 'PhoneId': 234, 'x': 601514.94, 'y': 650068.4},
 {'t': 0, 'PhoneId': 241, 'x': 601462.7, 'y': 648964.1},
 {'t': 0, 'PhoneId': 242, 'x': 601462.7, 'y': 648964.1},
 {'t': 0, 'PhoneId': 268, 'x': 600564.44, 'y': 652729.3},
 {'t': 0, 'PhoneId': 283, 'x': 602103.56, 'y': 649742.2},
 {'t': 0, 'PhoneId': 284, 'x': 602103.56, 'y': 649742.2},
 {'t': 0, 'PhoneId': 230, 'x': 601129.06, 'y': 653193.94},
 {'t': 0, 'PhoneId': 259, 'x': 600203.06, 'y': 651730.06},
 {'t': 0, 'PhoneId': 257, 'x': 600901.56, 'y': 651774.9},
 {'t': 0, 'PhoneId': 289, 'x': 600384.44, 'y': 650135.94},
 {'t': 0, 'PhoneId': 296, 'x': 600995.3, 'y': 652785.56},
 {'t': 0, 'PhoneId': 297, 'x': 600995.3, 'y': 652785.56},
 {'t': 0, 'PhoneId': 245, 'x': 600110.94, 'y': 651829.06},
 {'t': 0, 'PhoneId': 246, 'x': 600110.94, 'y': 651829.06},
 {'t': 0, 'PhoneId': 276, 'x': 601136.56, 'y': 649742.94},
 {'t': 0

In [49]:
import json
for i in range(0,main_df.select('t').rdd.max()[0]):
    main_df.select(main_df['t'] ==i).toJSON().map(lambda j: json.loads(j)).collect()\
    .writeStream.format("kafka").option("topic", "antennas")\
    .option("kafka.bootstrap.servers", "localhost:9092").start()
    

KeyboardInterrupt: 

In [50]:
      main_df.selectExpr("CAST(id AS STRING) AS key", "to_json(struct(*)) AS value").\
      writeStream.format("kafka").option("topic", "topicName")\
      .option("kafka.bootstrap.servers", "localhost:9092")\
      .option("checkpointLocation", "./chkpt").start()

AnalysisException: "cannot resolve '`id`' given input columns: [AntennaId, t, x, EventCode, TileId, y, PhoneId]; line 1 pos 5;\n'Project [cast('id as string) AS key#21032, structstojson(named_struct(t, t#9707, AntennaId, AntennaId#9708, EventCode, EventCode#9709, PhoneId, PhoneId#9710, x, x#9711, y, y#9712, TileId, TileId#9713), Some(Europe/Paris)) AS value#21033]\n+- Union\n   :- Relation[t#9707,AntennaId#9708,EventCode#9709,PhoneId#9710,x#9711,y#9712,TileId#9713] csv\n   :- Relation[t#9721,AntennaId#9722,EventCode#9723,PhoneId#9724,x#9725,y#9726,TileId#9727] csv\n   :- Relation[t#9742,AntennaId#9743,EventCode#9744,PhoneId#9745,x#9746,y#9747,TileId#9748] csv\n   :- Relation[t#9763,AntennaId#9764,EventCode#9765,PhoneId#9766,x#9767,y#9768,TileId#9769] csv\n   :- Relation[t#9784,AntennaId#9785,EventCode#9786,PhoneId#9787,x#9788,y#9789,TileId#9790] csv\n   :- Relation[t#9805,AntennaId#9806,EventCode#9807,PhoneId#9808,x#9809,y#9810,TileId#9811] csv\n   :- Relation[t#9826,AntennaId#9827,Eve

In [53]:
main_df.selectExpr("CAST(column AS STRING) AS key","to_json(struct(*)) AS value")

AnalysisException: "cannot resolve '`column`' given input columns: [AntennaId, t, x, EventCode, TileId, y, PhoneId]; line 1 pos 5;\n'Project [cast('column as string) AS key#21038, structstojson(named_struct(t, t#9707, AntennaId, AntennaId#9708, EventCode, EventCode#9709, PhoneId, PhoneId#9710, x, x#9711, y, y#9712, TileId, TileId#9713), Some(Europe/Paris)) AS value#21039]\n+- Union\n   :- Relation[t#9707,AntennaId#9708,EventCode#9709,PhoneId#9710,x#9711,y#9712,TileId#9713] csv\n   :- Relation[t#9721,AntennaId#9722,EventCode#9723,PhoneId#9724,x#9725,y#9726,TileId#9727] csv\n   :- Relation[t#9742,AntennaId#9743,EventCode#9744,PhoneId#9745,x#9746,y#9747,TileId#9748] csv\n   :- Relation[t#9763,AntennaId#9764,EventCode#9765,PhoneId#9766,x#9767,y#9768,TileId#9769] csv\n   :- Relation[t#9784,AntennaId#9785,EventCode#9786,PhoneId#9787,x#9788,y#9789,TileId#9790] csv\n   :- Relation[t#9805,AntennaId#9806,EventCode#9807,PhoneId#9808,x#9809,y#9810,TileId#9811] csv\n   :- Relation[t#9826,AntennaId#

In [ ]:
def actualisation(base, batch):
    
    base = base.union(batch.na.drop() \
                           .orderBy('t', ascending=False) \
                           .dropDuplicates(subset = ['PhoneId']))
    
    base = base.orderBy('t', ascending=False) \
               .coalesce(1) \
               .dropDuplicates(subset = ['PhoneId'])
    
    return base

df0 = actualisation(df0, df1)

tile_base = df0.groupBy('TileId').count()

In [10]:
from pykafka import KafkaClient
import json 
from datetime import datetime
import time
import uuid 
import pandas as pd

input_file= pd.read_csv('/home/cerezamo/projet_stream/simulator/mobile_data/kafka_minimal_df.csv')

client = KafkaClient(hosts="localhost:9092")
topic=client.topics['antennos']
producer = topic.get_sync_producer()

input_file = input_file[['t','PhoneId','x','y']]

input_file.PhoneId = input_file.PhoneId.astype(str)
input_file.x = input_file.x/1000
input_file.y = input_file.y/1000

def generate_checkpoint(input_file):
    for i in range(0,input_file.t.max()):
        input_file_topush = input_file[input_file.t==i]
        data = {'x' : [] , 'y' : [] , 'text' : []}
        
        j=0
        while j < len(input_file_topush):
            data['text'].append(input_file['PhoneId'][j])
            data['x'].append(input_file['x'][j]/1000)
            data['y'].append(input_file['y'][j]/1000)
            j+=1
            
        message = json.dumps(data)
        print(message)
        producer.produce(message.encode('ascii'))
        time.sleep(3)
        
generate_checkpoint(input_file)

In [ ]:
def generate_checkpoint(input_file):
    for i in range(0,input_file.t.max()):
        input_file_topush = input_file[input_file.t==i]
        message = input_file_topush.to_json(orient='records')
        print(message)
        producer.produce(message.encode('ascii'))
        time.sleep(3)
        
generate_checkpoint(input_file)

[{"t":0,"PhoneId":"296","x":600.995281445,"y":652.785571876},{"t":0,"PhoneId":"297","x":600.995281445,"y":652.785571876},{"t":0,"PhoneId":"289","x":600.384426337,"y":650.135919067},{"t":0,"PhoneId":"257","x":600.901554282,"y":651.774870814},{"t":0,"PhoneId":"259","x":600.203078702,"y":651.73008},{"t":0,"PhoneId":"230","x":601.129055069,"y":653.193950076},{"t":0,"PhoneId":"283","x":602.103592967,"y":649.742183146},{"t":0,"PhoneId":"284","x":602.103592967,"y":649.742183146},{"t":0,"PhoneId":"268","x":600.564455101,"y":652.729315347},{"t":0,"PhoneId":"241","x":601.462694847,"y":648.964107958},{"t":0,"PhoneId":"242","x":601.462694847,"y":648.964107958},{"t":0,"PhoneId":"234","x":601.514911596,"y":650.068348954},{"t":0,"PhoneId":"273","x":600.216462154,"y":649.458409534},{"t":0,"PhoneId":"291","x":600.50961435,"y":651.407658028},{"t":0,"PhoneId":"292","x":600.50961435,"y":651.407658028},{"t":0,"PhoneId":"270","x":600.008765087,"y":650.833277593},{"t":0,"PhoneId":"299","x":600.431357885,"y":

[{"t":5,"PhoneId":"242","x":601.15316618,"y":651.525617877},{"t":5,"PhoneId":"241","x":601.15316618,"y":651.525617877},{"t":5,"PhoneId":"230","x":600.660717662,"y":651.332288618},{"t":5,"PhoneId":"289","x":600.384426337,"y":650.135919067},{"t":5,"PhoneId":"257","x":600.242429502,"y":652.408177728},{"t":5,"PhoneId":"230","x":600.660717662,"y":651.332288618},{"t":5,"PhoneId":"284","x":601.326463487,"y":650.448298249},{"t":5,"PhoneId":"283","x":601.326463487,"y":650.448298249},{"t":5,"PhoneId":"253","x":600.091015093,"y":649.791594036},{"t":5,"PhoneId":"297","x":600.373529431,"y":653.390314852},{"t":5,"PhoneId":"296","x":600.373529431,"y":653.390314852},{"t":5,"PhoneId":"234","x":601.514911596,"y":650.068348954},{"t":5,"PhoneId":"257","x":600.242429502,"y":652.408177728},{"t":5,"PhoneId":"268","x":601.159144747,"y":650.051779337},{"t":5,"PhoneId":"242","x":601.15316618,"y":651.525617877},{"t":5,"PhoneId":"241","x":601.15316618,"y":651.525617877},{"t":5,"PhoneId":"273","x":600.19574499,"y"